In [12]:
import json

# Kodlama kuralları
kodlama = {
    "cinsiyet": {"Kadın": 1, "Erkek": 2},
    "egitim": {
        "OkuryazarDegil": 1, "Okuryazar": 2, "İlkokul": 3, "Ortaokul": 4,
        "Lise": 5, "Ön lisans": 6, "Lisans": 7, "Lisansüstü": 8
    },
    "medeni": {"Evli": 1, "Bekar": 2, "Dul": 3},
    "ekonomi": {"Asgari altı": 1, "Asgari": 2, "Asgari üstü": 3},
    "guvence": {"Var": 1, "Yok": 2},
    "calisma": {"Çalışıyor": 1, "Çalışmıyor": 2, "Emekli": 3},
    "kronik": {
        "Başka yok": 1, "Diyabet": 2, "Tansiyon": 3, "Astım": 4,
        "Hipertansiyon": 5, "Diğer": 6, "Hepsi": 7
    },
    "sigara": {"İçiyor": 1, "Bırakmış": 2, "Hiç içmemiş": 3},
    "ilac_kullanimi": {"Düzenli": 1, "NefesDarlıgında": 2, "Kullanmıyor": 3},
}

# JSON dosyasını okuma ve verileri işleme
def verileri_isle(input_path, output_path):
    # Dosyayı okuma
    with open(input_path, "r", encoding="utf-8") as file:
        veriler = json.load(file)

    islenmis_veriler = []
    eksik_veriler = []

    for veri in veriler:
        yeni_veri = {}
        eksik = False

        for alan, deger in veri.items():
            if alan in kodlama:
                if deger in kodlama[alan]:
                    yeni_veri[alan] = kodlama[alan][deger]
                else:
                    eksik = True  # Eksik veya yanlış değer
                    yeni_veri[alan] = None
            else:
                yeni_veri[alan] = deger  # Sayısallaştırma gerekmezse olduğu gibi ekle

        if eksik:
            eksik_veriler.append(veri["denek"])  # Eksik veriyi belirlemek için denek ID'si ekleniyor

        islenmis_veriler.append(yeni_veri)

    # Sayısal hale getirilmiş verileri yeni bir dosyaya kaydetme
    with open(output_path, "w", encoding="utf-8") as file:
        json.dump(islenmis_veriler, file, ensure_ascii=False, indent=4)

    return eksik_veriler

# Dosya yolları
input_path = "denekler.json"  # Giriş dosyasının yolu
output_path = "sayisal_veri.json"  # Çıkış dosyasının yolu

# Verileri işleyelim
eksik_veriler = verileri_isle(input_path, output_path)

# Eksik veriler hakkında bilgi verelim
if eksik_veriler:
    print("Eksik veya hatalı veriler tespit edildi. Eksik denek ID'leri:")
    print(eksik_veriler)
else:
    print("Tüm veriler eksiksiz işlendi.")


Tüm veriler eksiksiz işlendi.


In [16]:
import json

# Risk değerlendirme fonksiyonu
def risk_hesapla(denek):
    risk = 0

    # Cinsiyet
    if denek["cinsiyet"] == 1:  # Kadın
        risk += 2
    elif denek["cinsiyet"] == 2:  # Erkek
        risk += 1

    # Eğitim seviyesi
    risk += max(0, 8 - denek["egitim"])  # Eğitim seviyesi arttıkça risk azalır

    # Medeni durum
    if denek["medeni"] == 2:  # Bekar
        risk += 2
    elif denek["medeni"] == 1:  # Evli
        risk += 1

    # Ekonomik durum
    if denek["ekonomi"] == 1:  # Asgari altı
        risk += 3
    elif denek["ekonomi"] == 2:  # Asgari
        risk += 2
    elif denek["ekonomi"] == 3:  # Asgari üstü
        risk += 1

    # Güvence
    if denek["guvence"] == 2:  # Yok
        risk += 2

    # Kronik hastalık
    if denek["kronik"] == 1:  # Başka yok
        risk += 0
    elif denek["kronik"] in [2, 3, 4, 5, 6]:  # Diyabet, Tansiyon, vb.
        risk += 2
    elif denek["kronik"] == 7:  # Hepsi
        risk += 3

    # Sigara kullanımı
    if denek["sigara"] == 1:  # İçiyor
        risk += 3
    elif denek["sigara"] == 2:  # Bırakmış
        risk += 2
    elif denek["sigara"] == 3:  # Hiç içmemiş
        risk += 1

    # İlaç kullanımı
    if denek["ilac_kullanimi"] == 1:  # Düzenli
        risk += 2
    elif denek["ilac_kullanimi"] == 3:  # Kullanıyor
        risk += 0

    # Yaş
    if denek.get("yas", 0) > 40:  # Yaş 40'tan büyükse
        risk += 3

    return risk

# JSON dosyasını okuma
with open("sayisal_veri.json", "r", encoding="utf-8") as file:
    veriler = json.load(file)

# Maksimum risk puanı
maksimum_risk = 27  # Yaş kriteri eklendiği için güncellendi

# Her denek için yüzdelik risk hesaplama
for denek in veriler:
    denek_id = denek.get("denek", "Bilinmiyor")
    risk_orani = risk_hesapla(denek)
    yuzdelik_risk = (risk_orani / maksimum_risk) * 100
    print(f"Denek ID: {denek_id}, KOAH Risk Oranı: {risk_orani} ({yuzdelik_risk:.2f}%)")


Denek ID: 1, KOAH Risk Oranı: 20 (74.07%)
Denek ID: 2, KOAH Risk Oranı: 14 (51.85%)
Denek ID: 3, KOAH Risk Oranı: 15 (55.56%)
Denek ID: 4, KOAH Risk Oranı: 18 (66.67%)
Denek ID: 5, KOAH Risk Oranı: 17 (62.96%)
Denek ID: 6, KOAH Risk Oranı: 19 (70.37%)
Denek ID: 7, KOAH Risk Oranı: 19 (70.37%)
Denek ID: 8, KOAH Risk Oranı: 17 (62.96%)
Denek ID: 9, KOAH Risk Oranı: 19 (70.37%)
Denek ID: 10, KOAH Risk Oranı: 21 (77.78%)
Denek ID: 11, KOAH Risk Oranı: 19 (70.37%)
Denek ID: 12, KOAH Risk Oranı: 18 (66.67%)
Denek ID: 13, KOAH Risk Oranı: 14 (51.85%)
Denek ID: 14, KOAH Risk Oranı: 17 (62.96%)
Denek ID: 15, KOAH Risk Oranı: 20 (74.07%)
Denek ID: 16, KOAH Risk Oranı: 19 (70.37%)
Denek ID: 17, KOAH Risk Oranı: 20 (74.07%)
Denek ID: 18, KOAH Risk Oranı: 18 (66.67%)
Denek ID: 19, KOAH Risk Oranı: 18 (66.67%)
Denek ID: 20, KOAH Risk Oranı: 15 (55.56%)
Denek ID: 21, KOAH Risk Oranı: 14 (51.85%)
Denek ID: 22, KOAH Risk Oranı: 17 (62.96%)
Denek ID: 23, KOAH Risk Oranı: 17 (62.96%)
Denek ID: 24, KOAH R

In [17]:
import json

# JSON dosyasını okuma
with open("sayisal_veri.json", "r", encoding="utf-8") as file:
    veriler = json.load(file)

# Meslek başlığını çıkarma
for denek in veriler:
    denek.pop("meslek", None)

# Yeni JSON dosyasına kaydetme
with open("sayisal_veri_sayisal.json", "w", encoding="utf-8") as file:
    json.dump(veriler, file, ensure_ascii=False, indent=4)


In [21]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Uyarıları bastırma
warnings.filterwarnings("ignore", category=UserWarning)

# Verileri JSON'dan yükleme
with open("sayisal_veri_sayisal.json", "r", encoding="utf-8") as file:
    veriler = json.load(file)

# Veriyi DataFrame'e dönüştürme
df = pd.DataFrame(veriler)

# KOAH riskini hedef değişken olarak belirleme
# Risk puanını hesaplama fonksiyonu
def risk_hesapla(denek):
    risk = 0
    # Cinsiyet
    if denek["cinsiyet"] == 1:  # Kadın
        risk += 2
    elif denek["cinsiyet"] == 2:  # Erkek
        risk += 1
    # Eğitim seviyesi
    risk += max(0, 8 - denek["egitim"])  # Eğitim seviyesi arttıkça risk azalır
    # Medeni durum
    if denek["medeni"] == 2:  # Bekar
        risk += 2
    elif denek["medeni"] == 1:  # Evli
        risk += 1
    # Ekonomik durum
    if denek["ekonomi"] == 1:  # Asgari altı
        risk += 3
    elif denek["ekonomi"] == 2:  # Asgari
        risk += 2
    elif denek["ekonomi"] == 3:  # Asgari üstü
        risk += 1
    # Güvence
    if denek["guvence"] == 2:  # Yok
        risk += 2
    # Kronik hastalık
    if denek["kronik"] == 1:  # Başka yok
        risk += 0
    elif denek["kronik"] in [2, 3, 4, 5, 6]:  # Diyabet, Tansiyon, vb.
        risk += 2
    elif denek["kronik"] == 7:  # Hepsi
        risk += 3
    # Sigara kullanımı
    if denek["sigara"] == 1:  # İçiyor
        risk += 3
    elif denek["sigara"] == 2:  # Bırakmış
        risk += 2
    elif denek["sigara"] == 3:  # Hiç içmemiş
        risk += 1
    # İlaç kullanımı
    if denek["ilac_kullanimi"] == 1:  # Düzenli
        risk += 2
    elif denek["ilac_kullanimi"] == 3:  # Kullanıyor
        risk += 0
    return risk

# Her denek için risk hesaplama ve hedef değişken oluşturma
df["risk_puani"] = df.apply(risk_hesapla, axis=1)
df["KOAH"] = (df["risk_puani"] > 12).astype(int)  # Örnek eşik değeri: 12

# Model için özellikler ve hedef değişkeni ayırma
X = df.drop(columns=["denek", "risk_puani", "KOAH"])
y = df["KOAH"]

# Verileri eğitim ve test seti olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model seçimi ve eğitimi
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)

# Model performansını değerlendirme
print("Doğruluk Skoru:", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred, zero_division=1))

# Yeni bir denek için tahmin
yeni_denek = {
    "yas": 45,
    "cinsiyet": 1,
    "egitim": 4,
    "medeni": 2,
    "ekonomi": 2,
    "guvence": 1,
    "calisma": 3,
    "kronik": 2,
    "sigara": 1,
    "ilac_kullanimi": 1,
}
yeni_denek_df = pd.DataFrame([yeni_denek])
tahmin = model.predict(yeni_denek_df)
print(f"Yeni denek KOAH riski: {'Evet' if tahmin[0] == 1 else 'Hayır'}")


Doğruluk Skoru: 0.8333333333333334

Sınıflandırma Raporu:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.83      0.91         6

    accuracy                           0.83         6
   macro avg       0.50      0.42      0.45         6
weighted avg       1.00      0.83      0.91         6

Yeni denek KOAH riski: Evet


c:\Users\egela\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\egela\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\egela\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita